In [1]:
import time
import email
import pickle

import os; 
from os import listdir
from os.path import join, isdir

import pandas as pd
import swifter

import matplotlib.pyplot as plt

path = "/media/becode/3D_House/Enron_CSV/"

In [3]:
all_mails = pd.read_csv(join(path, "all_mails.csv"), index_col=0)
employee = pd.read_csv(join(path, "enron_employees.csv"), index_col=0)

In [16]:
mails_lookup = {
    employ: employee.loc[idx, "Mails"]
    for idx, employ in enumerate(list(employee["Employ"]))
}

In [49]:
employ = "allen-p"
df = all_mails[all_mails["X-Origin"]== employ]
df = df[df["From"].apply(lambda x:x in (mails_lookup[employ]))]
df = df[["Content", "Date", "From", "To", "Subject"]]

In [58]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet

tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))

def get_lemma(word):
    lemma = wordnet.morphy(word)
    if lemma is None: return word
    else: return lemma

import en_core_web_lg
eng = en_core_web_lg.load()
vectorize = lambda x: [eng.vocab[w].vector for w in x]

## Reduce all mails to sent only

In [121]:
dfs = []
abs_start = start = time.time()
for idx, user in enumerate(all_mails["X-Origin"].unique()):
    df = all_mails[all_mails["X-Origin"]== user]
    df = df[df["From"].apply(lambda x:x in (mails_lookup[user]))]
    df = df[["Content", "Date", "From", "To", "Subject"]]
    df["Origin"] = [user for i in range(len(df))]
    
    tokens = list(df["Content"].apply(tokenizer.tokenize))
    tokens = [[token.lower() for token in tokenenized if len(token) > 4]
              for tokenenized in tokens]
    tokens = [[token for token in tokenenized if token not in stop_words]
                    for tokenenized in tokens]
    df["Tokens"] = [[get_lemma(token) for token in tokenenized]
                    for tokenenized in tokens]
    
    df.drop(df[df["Tokens"].apply(lambda x: len(x) > 10000)].index, inplace=True)
    
    df["VecSum"] = df["Tokens"].apply(lambda x:
        sum(vectorize(x)).sum() if type(sum(vectorize(x))) != int else sum(vectorize(x)))
    
    drop_idxs = []
    for vec in df["VecSum"].unique():
        similar = df[df["VecSum"] == vec]
        for date in similar["Date"].unique():
            drop_idxs.extend(similar[similar["Date"] == date].index[1:])
    
    df.drop(drop_idxs, inplace=True)
    df["Origin"] = [user for i in range(len(df))]
    dfs.append(df); del(df)
    
    print(idx +1, "/", len(all_mails["X-Origin"].unique()),
          user, "took", round(time.time() -start, 2), "seconds.")
    start = time.time()
    
sent_mails = pd.concat(dfs)
print("Total time taken:", round(time.time() -abs_start, 2)/60, "minutes.")

del(dfs)
sent_mails

1 / 150 allen-p took 4.33 seconds.
2 / 150 arnold-j took 7.88 seconds.
3 / 150 arora-h took 0.37 seconds.
4 / 150 badeer-r took 0.47 seconds.
5 / 150 bailey-s took 0.59 seconds.
6 / 150 bass-e took 8.61 seconds.
7 / 150 baughman-d took 0.44 seconds.
8 / 150 beck-s took 9.76 seconds.
9 / 150 benson-r took 0.21 seconds.
10 / 150 blair-l took 3.83 seconds.
11 / 150 brawner-s took 1.19 seconds.
12 / 150 buy-r took 1.99 seconds.
13 / 150 campbell-l took 3.48 seconds.
14 / 150 carson-m took 0.9 seconds.
15 / 150 cash-m took 5.62 seconds.
16 / 150 causholli-m took 1.41 seconds.
17 / 150 corman-s took 2.04 seconds.
18 / 150 crandell-s took 0.81 seconds.
19 / 150 cuilla-m took 0.67 seconds.
20 / 150 dasovich-j took 30.18 seconds.
21 / 150 davis-d took 1.42 seconds.
22 / 150 dean-c took 0.13 seconds.
23 / 150 delainey-d took 6.74 seconds.
24 / 150 derrick-j took 2.95 seconds.
25 / 150 dickson-s took 0.62 seconds.
26 / 150 donoho-l took 0.65 seconds.
27 / 150 donohoe-t took 0.17 seconds.
28 / 150

,Content,Date,From,To,Subject,Tokens,VecSum,Origin
254576,---------------------- Forwarded by Phillip K ...,2000-09-01 06:08:00,phillip.allen@enron.com,"mike.grigsby@enron.com, frank.ermis@enron.com",FYI,"[forward, phillip, allen, enron, north, americ...",7.766356,allen-p
254452,"Naomi,\n\nThe two analysts that I have had con...",1999-12-10 07:00:00,phillip.allen@enron.com,naomi.johnston@enron.com,NaN,"[naomi, analyst, contact, lenhart, vishal, rep...",-14.676489,allen-p
254553,---------------------- Forwarded by Phillip K ...,2000-02-11 04:31:00,phillip.allen@enron.com,keith.holst@enron.com,RE: W basis quotes,"[forward, phillip, allen, george, rahal, georg...",61.238914,allen-p
254418,---------------------- Forwarded by Phillip K ...,2000-02-11 07:39:00,phillip.allen@enron.com,mike.grigsby@enron.com,Western Strategy Briefing,"[forward, phillip, allen, heizenrader, james, ...",-8.750636,allen-p
255431,Let me know when you get the quotes from Pauli...,2001-05-11 11:26:00,phillip.allen@enron.com,jsmith@austintx.com,NaN,"[quote, pauline, expect, something, range, wou...",16.326239,allen-p
...,...,...,...,...,...,...,...,...
48625,Can you re-activate the Alberta Power Pool Acc...,2001-08-08 10:10:42,john.zufferli@enron.com,help@enrononline.com,Power Pool of Alberta EnronOnline Access,"[activate, alberta, power, access, enrononline...",177.460449,zufferli-j
48443,"\n\n-----Original Message-----\nFrom: Lalani, ...",2001-08-08 12:39:36,john.zufferli@enron.com,cooper.richey@enron.com,"FW: lavo_may_10 (slides 13-16)-revised (Aug 7,...","[original, message, lalani, tuesday, august, z...",12.110354,zufferli-j
48522,Enron Canada Power Corp acknowledges the chang...,2002-01-09 07:30:16,john.zufferli@enron.com,carol.moline@powerpool.ab.ca,Chevron Contract,"[enron, canada, power, acknowledge, change, as...",43.984802,zufferli-j
48586,Nella here is a list of products that we would...,2002-01-09 09:46:43,john.zufferli@enron.com,nella.cappelletto@enron.com,Products,"[nella, product, would, launch, netco, assume,...",-2.680151,zufferli-j


In [68]:
sent_mails.to_csv(join(path, "sent_mails.csv"))

## Dictionary & corpus

In [55]:
import gensim
from gensim import corpora

In [122]:
start = time.time()

collection = []
for idx in sent_mails.index:
    collection.append(sent_mails.loc[idx, "Tokens"])
    
print("Took", round(time.time()-start, 2), "seconds.")

Took 0.85 seconds.


In [123]:
start = time.time()

dictionary = corpora.Dictionary(collection)
dictionary.save(join(path, "sent_dictionary.gensim"))

print("Took", round(time.time()-start, 2), "seconds.")

Took 12.51 seconds.


In [124]:
print(len(dictionary.keys()), "unique words in dictionary")

155872 unique words in dictionary


In [125]:
start = time.time()

corpus = [dictionary.doc2bow(tokens) for tokens in collection]
pickle.dump(corpus, open(join(path, "sent_corpus.pkl"), "wb"))

print("Took", round(time.time()-start, 2), "seconds.")

Took 8.22 seconds.


In [126]:
sent_mails["Corpus"] = corpus

## Replied

In [95]:
reply_lst = ["RE","Re:","re:"]

In [127]:
sent_mails["Reply"] = sent_mails["Subject"].swifter.apply(
    lambda x: any(map(lambda tag: tag in x, reply_lst))
    if type(x) != float else False)

# Forward

In [128]:
dict_reverse = {v:k for k,v in dictionary.items()}

In [199]:
forward_word_idxs = list(map(lambda x: dict_reverse[x], ["forward", "forwarding", "dforward","forwarde"]))

In [201]:
forward_idxs = sent_mails[sent_mails["Corpus"].swifter.apply(lambda x: any([
        any(map(lambda pair: pair[0]==word, x))
        for word in forward_word_idxs
]))].index

In [210]:
sent_mails["Forward"] = [True if idx in forward_idxs else False for idx in sent_mails.index]

## Reordered and save

In [214]:
sent_mails = sent_mails[
    ["Date","Origin","From","To","Subject","Forward","Reply","Content","Tokens","Corpus","VecSum"]
]; sent_mails.to_csv(join(path, "sent_mails.csv"))